# Was ist ein LSTM-Modell und wie funktioniert es?

Ein **Long Short-Term Memory (LSTM)** Netzwerk ist eine spezielle Art von rekurrenten neuronalen Netzwerken (RNNs), die besonders gut für die Verarbeitung von Sequenzdaten geeignet ist. Im Gegensatz zu klassischen RNNs können LSTMs langfristige Abhängigkeiten in Daten lernen und das Problem des verschwindenden Gradienten (Vanishing Gradient Problem) lösen. Das macht LSTMs sehr leistungsfähig für Aufgaben wie Zeitreihenanalyse, Textverarbeitung und andere sequenzielle Klassifikationsaufgaben.

## Basic LSTM-Architektur

```python
class BasicLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.2, bidirectional=False):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional
        )
        
        # Calculate the size of the fully connected layer
        fc_input_size = hidden_size * 2 if bidirectional else hidden_size
        
        self.fc = nn.Sequential(
            nn.Linear(fc_input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, num_classes)
        )
    
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out
```

**Erklärung der Basic LSTM-Architektur:**
- Die LSTM-Schicht (`nn.LSTM`) verarbeitet die Eingabesequenz und lernt zeitliche Abhängigkeiten. Die Parameter `input_size`, `hidden_size` und `num_layers` bestimmen die Eingabedimension, die Grösse der versteckten Zustände und die Anzahl der LSTM-Schichten.
- **Bidirektionale Option**: Wenn `bidirectional=True`, verarbeitet das LSTM die Sequenz in beide Richtungen (vorwärts und rückwärts), was die Kontextinformationen verbessert.
- **Dropout-Regularisierung**: Wird nur angewendet, wenn mehr als eine Schicht vorhanden ist, um Overfitting zu vermeiden.
- **Vollverbundene Schichten**: Am Ende werden die LSTM-Ausgaben durch zwei lineare Schichten auf die gewünschte Anzahl von Klassen abgebildet.
- **Letzter Zeitschritt**: Das Modell verwendet nur den letzten Zeitschritt (`out[:, -1, :]`) für die finale Klassifikation.

---



## Advanced LSTM mit Attention-Mechanismus

```python
class AdvancedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.2, bidirectional=False):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional
        )
        
        # Calculate the size of the fully connected layer
        fc_input_size = hidden_size * 2 if bidirectional else hidden_size
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(fc_input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(fc_input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_classes)
        )
    
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        lstm_out, _ = self.lstm(x, (h0, c0))
        
        # Attention mechanism
        attention_weights = self.attention(lstm_out)
        attention_weights = torch.softmax(attention_weights, dim=1)
        context = torch.sum(attention_weights * lstm_out, dim=1)
        
        # Final classification
        out = self.fc(context)
        return out
```

**Verbesserungen des Advanced LSTM:**
- **Attention-Mechanismus**: Statt nur den letzten Zeitschritt zu verwenden, berechnet das Modell Aufmerksamkeitsgewichte für alle Zeitschritte. Dadurch kann es automatisch die wichtigsten Teile der Sequenz identifizieren und fokussieren.
- **Gewichteter Kontext**: Der finale Kontext-Vektor wird als gewichteter Durchschnitt aller LSTM-Ausgaben berechnet, wodurch Informationen aus der gesamten Sequenz genutzt werden.
- **Tiefere Klassifikationsschicht**: Drei vollverbundene Schichten mit abnehmender Grösse ermöglichen komplexere Entscheidungsgrenzen.
- **Bessere Informationsnutzung**: Alle Zeitschritte tragen zur finalen Vorhersage bei, nicht nur der letzte.

---

## Warum haben wir diese "Tweaks" verwendet?

- **Bidirektionale Verarbeitung**: Ermöglicht dem Modell, sowohl vergangene als auch zukünftige Kontextinformationen zu nutzen, was besonders bei Sequenzklassifikation vorteilhaft ist.
- **Batch First (`batch_first=True`)**: Erleichtert die Handhabung der Tensoren, da die Batch-Dimension an erster Stelle steht.
- **Adaptive Dropout**: Dropout wird nur bei mehrschichtigen LSTMs angewendet, da es bei einschichtigen LSTMs zu Informationsverlust führen kann.
- **Attention-Mechanismus**: Löst das Problem, dass bei langen Sequenzen wichtige Informationen vom Anfang der Sequenz verloren gehen können.
- **Graduelle Dimensionsreduktion**: In der Advanced-Version werden die Dimensionen schrittweise reduziert (hidden_size → hidden_size//2 → num_classes), was zu stabilerem Training führt.
- **Hyperparameter-Tuning**: Parameter wie `hidden_size`, `num_layers`, `dropout` und `bidirectional` können im Grid Search optimiert werden, um die beste Modellkonfiguration für spezifische Datensätze zu finden.

Insgesamt sorgen diese Anpassungen dafür, dass die LSTM-Modelle sowohl leistungsfähig als auch robust gegenüber verschiedenen Arten von Sequenzdaten sind.

# Was ist ein MLP-Modell und wie funktioniert es?

Ein **Multi-Layer Perceptron (MLP)** ist ein feedforward-Netzwerk, das aus mehreren vollverbundenen Schichten (Dense Layers) besteht. Im Gegensatz zu CNNs oder LSTMs hat ein MLP keine spezielle Struktur für räumliche oder zeitliche Daten, sondern verarbeitet die Eingaben als flache Vektoren. MLPs sind sehr vielseitig und eignen sich hervorragend für tabellarische Daten, allgemeine Klassifikationsaufgaben und als finale Klassifikationsschichten in komplexeren Architekturen.

## Basic MLP-Architektur

```python
class BasicMLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, dropout=0.2):
        super().__init__()
        layers = []
        prev_size = input_size
        
        # Add hidden layers
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_size = hidden_size
        
        # Add output layer
        layers.append(nn.Linear(prev_size, num_classes))
        
        self.mlp = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.mlp(x)
```

**Erklärung der Basic MLP-Architektur:**
- **Flexible Schichtgrössen**: Die Liste `hidden_sizes` bestimmt die Anzahl und Grösse der versteckten Schichten. Zum Beispiel `[128, 64, 32]` erstellt drei versteckte Schichten mit abnehmender Grösse.
- **Lineare Transformationen**: Jede Schicht führt eine lineare Transformation (`nn.Linear`) durch, die die Eingaben mit Gewichten multipliziert und Bias addiert.
- **Batch Normalization**: Normalisiert die Aktivierungen für stabileres und schnelleres Training.
- **ReLU-Aktivierung**: Die Rectified Linear Unit-Funktion führt Nicht-Linearität ein und hilft beim Gradientenfluss.
- **Dropout-Regularisierung**: Reduziert Overfitting durch zufälliges "Ausschalten" von Neuronen während des Trainings.
- **Modularer Aufbau**: Die Schichten werden dynamisch basierend auf der `hidden_sizes`-Liste erstellt.

---



## Advanced MLP mit Residual Connections

```python
class AdvancedMLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, dropout=0.2):
        super().__init__()
        self.input_size = input_size
        
        # Feature extraction layers
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]),
            nn.BatchNorm1d(hidden_sizes[0]),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # Residual blocks
        self.residual_blocks = nn.ModuleList()
        for i in range(len(hidden_sizes)-1):
            self.residual_blocks.append(
                ResidualBlock(hidden_sizes[i], hidden_sizes[i+1], dropout)
            )
        
        # Output layer
        self.classifier = nn.Sequential(
            nn.Linear(hidden_sizes[-1], num_classes)
        )
    
    def forward(self, x):
        x = self.feature_extractor(x)
        for block in self.residual_blocks:
            x = block(x)
        return self.classifier(x)

class ResidualBlock(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.2):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(out_features, out_features),
            nn.BatchNorm1d(out_features)
        )
        self.shortcut = nn.Linear(in_features, out_features) if in_features != out_features else nn.Identity()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        residual = self.shortcut(x)
        out = self.block(x)
        out += residual
        out = self.relu(out)
        out = self.dropout(out)
        return out
```

**Verbesserungen des Advanced MLP:**
- **Residual Connections (Skip Connections)**: Ermöglichen das Training tieferer Netzwerke durch Umgehung des Vanishing Gradient Problems. Die Eingabe wird direkt zur Ausgabe addiert.
- **Modularer Aufbau**: Getrennte Feature-Extraktion und Residual Blocks für bessere Strukturierung und Verständlichkeit.
- **ResidualBlock-Klasse**: Implementiert das bewährte Residual-Konzept aus ResNet-Architekturen für MLPs.
- **Adaptive Shortcut-Verbindungen**: Wenn sich die Dimensionen zwischen Eingabe und Ausgabe ändern, wird eine lineare Transformation verwendet, ansonsten eine Identity-Funktion.
- **Doppelte Transformation**: Jeder Residual Block führt zwei lineare Transformationen durch, bevor die Residual-Verbindung addiert wird.

---

## Warum haben wir diese "Tweaks" verwendet?

- **Batch Normalization**: Stabilisiert das Training durch Normalisierung der Aktivierungen, reduziert die interne Kovariatenverschiebung und ermöglicht höhere Lernraten.
- **ReLU-Aktivierung**: Löst das Problem verschwindender Gradienten besser als Sigmoid oder Tanh-Funktionen und ist computationally effizient.
- **Dropout-Regularisierung**: Verhindert Overfitting durch zufälliges Deaktivieren von Neuronen, was das Modell zwingt, robustere Features zu lernen.
- **Flexible Architektur**: Die `hidden_sizes`-Liste ermöglicht einfache Anpassung der Netzwerktiefe und -breite für verschiedene Problemgrössen.
- **Residual Connections**: Ermöglichen das Training sehr tiefer Netzwerke und verbessern den Gradientenfluss, was zu besserer Konvergenz führt.
- **Sequential-Aufbau**: Durch `nn.Sequential` wird der Code sauberer und die Architektur ist einfacher zu verstehen und zu modifizieren.
- **Hyperparameter-Tuning**: Parameter wie `hidden_sizes`, `dropout`, Lernrate und Batch-Grösse können im Grid Search optimiert werden.

## ResidualBlock im Detail

Der ResidualBlock implementiert die Formel: **F(x) = H(x) + x**, wobei:
- **H(x)** die Transformation durch die beiden linearen Schichten ist
- **x** die ursprüngliche Eingabe (Residual/Skip Connection)
- **F(x)** die finale Ausgabe nach Addition und Aktivierung

Dies ermöglicht es dem Netzwerk, Identitätsfunktionen leichter zu lernen und tiefere Architekturen zu trainieren.
Insgesamt sorgen diese Anpassungen dafür, dass die MLP-Modelle sowohl für einfache als auch für komplexe Klassifikationsaufgaben optimal geeignet sind.